thread和task两种切换顺序的不同方式，分别对应Python中并发的两种形式---threading和asyncio。

**并发编程之Futures**

In [ ]:
import concurrent.futures
import requests
import threading
import time

def download_one(url):
    resp = requests.get(url)
    print('Read {} characters from {}'.format(len(resp.content), url))

def download_all(urls):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_one, urls)

def main():
    sites = [ 
        'https://www.baidu.com/', 
        'https://pypi.org/', 
        'https://www.sina.com.cn/', 
        'https://www.163.com/', 
        'https://news.qq.com/', 
        'http://www.ifeng.com/', 
        'http://www.ce.cn/', 
        'https://news.baidu.com/', 
        'http://www.people.com.cn/', 
        'http://www.ce.cn/', 
        'https://news.163.com/', 
        'http://news.sohu.com/' 
    ]
    start_time = time.perf_counter()
    download_all(sites)
    end_time = time.perf_counter()
    print('Downloads {} sites in {} seconds'.format(len(sites), end_time - start_time))
    

main()

In [12]:
print(len('ab'))

2


Futures中还有一个重要的函数result()，它表示当future完成后，返回其对应的结果或异常。而as_completed(fs)，则是针对给定的future迭代器fs，在其完成后，返回完成的迭代器。

In [19]:
from concurrent import futures
import time
import random

def returnNumber(number: int):
    print("start returnNumber {}".format(number))
    time.sleep(random.randint(10, 20))
    print("end returnNumber {}".format(number))
    return number

if __name__ == "__main__":
    with futures.ThreadPoolExecutor(5) as executor:
        to_do = []
        for number in range(0, 10):
            future = executor.submit(returnNumber, number) # 添加到futures的任务队列，等待排队执行
            print(future)
            to_do.append(future)
        print("---middle---")
        for future in futures.as_completed(to_do): # 当某一个future任务执行完毕，执行下面的代码。会堵塞，等待线程完成后执行
            res = future.result() # 获取线程的返回结果
            print(res)
    print("end")
                 

start returnNumber 0<Future at 0x7ff3cd0b1ee0 state=running>

start returnNumber 1
<Future at 0x7ff3cd550c70 state=running>
start returnNumber 2
<Future at 0x7ff3cd550e80 state=running>
start returnNumber 3
<Future at 0x7ff3cd53edc0 state=running>
start returnNumber 4
<Future at 0x7ff3cbe70a60 state=running>
<Future at 0x7ff3cd550d90 state=pending>
<Future at 0x7ff3cd54fd90 state=pending>
<Future at 0x7ff3cd54fac0 state=pending>
<Future at 0x7ff3cd54f880 state=pending>
<Future at 0x7ff3cd54f1f0 state=pending>
---middle---
end returnNumber 3
start returnNumber 5
3
end returnNumber 0
start returnNumber 6
0
end returnNumber 4
start returnNumber 7
4
end returnNumber 1end returnNumber 2
start returnNumber 8
2

start returnNumber 9
1
end returnNumber 5
5
end returnNumber 7
7
end returnNumber 9
9
end returnNumber 6
6
end returnNumber 8
8
end


In [21]:
import concurrent.futures
import requests
import time

def download_one(url):
    resp = requests.get(url)
    print('Read {} from {}'.format(len(resp.content), url))

def download_all(sites):
    with futures.ThreadPoolExecutor(max_workers=5) as executor:
        to_do = []
        for site in sites:
            future = executor.submit(download_one, site)
            to_do.append(future)
        
        for future in to_do:
            res = future.result()
            print("res: {}".format(res))

def main():
    sites = [ 
        'https://www.baidu.com/', 
        'https://pypi.org/', 
        'https://www.sina.com.cn/', 
        'https://www.163.com/', 
        'https://news.qq.com/', 
        'http://www.ifeng.com/', 
        'http://www.ce.cn/', 
        'https://news.baidu.com/', 
        'http://www.people.com.cn/', 
        'http://www.ce.cn/', 
        'https://news.163.com/', 
        'http://news.sohu.com/' 
    ]
    start_time = time.perf_counter()
    download_all(sites)
    end_time = time.perf_counter()
    print('Download {} sites in {} seconds'.format(len(sites), end_time - start_time))

if __name__ == '__main__':
    main()

Read 2443 from https://www.baidu.com/
res: None
Read 7751 from https://news.qq.com/
Read 613992 from https://www.163.com/
Read 346384 from http://www.ifeng.com/
Read 120284 from http://www.ce.cn/
Read 493100 from https://www.sina.com.cn/
Read 98517 from http://www.people.com.cn/
Read 151882 from http://news.sohu.com/
Read 120284 from http://www.ce.cn/
Read 79199 from https://news.baidu.com/
Read 328436 from https://news.163.com/
Read 28698 from https://pypi.org/
res: None
res: None
res: None
res: None
res: None
res: None
res: None
res: None
res: None
res: None
res: None
Download 12 sites in 0.8258417280012509 seconds


**总结**

* 并发，通过线程和任务之间相互切换的方式实现，但同一时刻，只允许有一个线程或任务执行。
* 而并行，则是只多个进程同时执行。

并发通常用于I/O操作频繁的场景，而并行则适用于CPU heavy的场景。